# Classification Performance Dashboards for Diabetes Dataset

In [1]:
import pandas as pd
from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab, ClassificationPerformanceTab, CatTargetDriftTab



In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Dataset

In [3]:
dataset = pd.read_csv('../rfcmodel/diabetes.csv', sep=',')

## Data Drift

In [4]:
data_drift_report = Dashboard(tabs=[DataDriftTab])
data_drift_report.calculate(dataset[:75], dataset[75:], column_mapping = None)
data_drift_report.show()

## Model Performance Dashboard

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
reference, production, y_train, y_test = train_test_split(dataset.loc[:, dataset.columns.difference(['Outcome'])],
                                                        dataset.loc[:, 'Outcome'].values, test_size=0.25,
                                                        random_state=11)

target_y = dataset.loc[:, 'Outcome'].values

model = RandomForestClassifier(
            n_estimators=100,
            max_depth=3,
            max_features='log2',
            n_jobs=3
    )

model.fit(reference, y_train)

train_predictions = model.predict(reference)
test_predictions = model.predict(production)


reference['target'] = y_train
reference['prediction'] = train_predictions

production['target'] = y_test
production['prediction'] = test_predictions

feature_names = dataset.columns
feature_names = feature_names[:-1]

column_mapping = {}

column_mapping['target'] = 'target'
column_mapping['prediction'] = 'prediction'
column_mapping['numerical_features'] = feature_names

model_performance_dashboard = Dashboard(tabs=[ClassificationPerformanceTab])
model_performance_dashboard.calculate(reference, production, column_mapping = None)
model_performance_dashboard.show()


## Data and Target Drift

In [10]:
import numpy as np
dataset['target'] = target_y
dataset['prediction'] = np.concatenate((train_predictions, test_predictions))
data_and_target_drift_report = Dashboard(tabs=[DataDriftTab, CatTargetDriftTab])
data_and_target_drift_report.calculate(dataset[:75], dataset[75:], column_mapping = column_mapping)
data_and_target_drift_report.show()